# Create Tool AGI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/CreateTool-AGI/blob/main/examples/createtoolagi.ipynb)

In [1]:
!pip install git+https://github.com/fuyu-quant/CreateTool-AGI.git

  Cloning https://github.com/fuyu-quant/CreateTool-AGI.git to /tmp/pip-req-build-0y8kcnfa
  Running command git clone --filter=blob:none --quiet https://github.com/fuyu-quant/CreateTool-AGI.git /tmp/pip-req-build-0y8kcnfa
  Resolved https://github.com/fuyu-quant/CreateTool-AGI.git to commit a47059a3a0508bdd4ba570caf6ee09131b5bbd84
  Preparing metadata (setup.py) ... done


In [2]:
%%capture
!pip install langchain==0.0.167
!pip install openai==0.27.4

In [3]:
from createtoolagi import CreateToolAGI

import os
#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

## Normal LLM

In [ ]:
from langchain.llms import OpenAI

input = 'Count the number of prime numbers from 1 to 50 and multiply that number by 100.'

llm = OpenAI(temperature=0, model_name = 'gpt-4')

llm(input)

## Execute Create Tool AGI

In [5]:
input = 'Sum of prime numbers from 1 to 50'
tools = []
ctagi = CreateToolAGI()

ctagi.run(input, tools)

> Generalize the input task.
Generalized Task：Sum of prime numbers from A to B

> Determine if you should make a tool.
I must create the tool before executing.

> Create a tool.
Completed!
Created tool name：SumOfPrimesTool



> Entering new AgentExecutor chain...
I need to find the sum of prime numbers between 1 and 50. I can use the SumOfPrimesTool for this task.
Action: SumOfPrimesTool
Action Input: 1,50
Observation: 328
Thought:I now know the final answer.
Final Answer: 328

> Finished chain.


In [6]:
from langchain.tools import BaseTool

class SumOfPrimesTool(BaseTool):
    name = "SumOfPrimesTool"
    description = "used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 2 and 10, the input is '2,10'."

    def is_prime(self, num: int) -> bool:
        "Check if a number is prime."
        if num < 2:
            return False
        for i in range(2, num):
            if num % i == 0:
                return False
        return True

    def _run(self, query: str) -> str:
        "Use the tool."
        a, b = query.split(",")
        a, b = int(a), int(b)
        prime_sum = sum([x for x in range(a, b + 1) if self.is_prime(x)])
        return prime_sum

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("SumOfPrimesTool does not support async")

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent


llm = OpenAI(temperature=0, model_name = 'gpt-4')

tools = [SumOfPrimesTool()]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

input = 'Sum of prime numbers from 1 to 2000'

agent.run(input)

## Export to file ver

In [12]:
input = 'Sum of prime numbers from 1 to 50'
tools = []
folder_path = '/home/CreateTool-AGI/data/'

ctagi = CreateToolAGI()


ctagi.run(input, tools, folder_path)

> Generalize the input task.
Generalized Task：Sum of prime numbers from A to B

> Determine if you should make a tool.
I must create the tool before executing.

> Create a tool.
Completed!
Created tool name：SumOfPrimesTool



> Entering new AgentExecutor chain...
I need to find the sum of prime numbers between 1 and 50. I can use the SumOfPrimesTool for this.
Action: SumOfPrimesTool
Action Input: 1,50
Observation: 328
Thought:I now know the final answer.
Final Answer: 328

> Finished chain.


## use the tool you created

In [15]:
from SumOfPrimesTool import SumOfPrimesTool

In [16]:
# Change inputs to new ones
input = 'Sum of prime numbers from 8 to 15'

# Use the tool you created
tools = [SumOfPrimesTool()]

ctagi = CreateToolAGI()

ctagi.run(input, tools)

> Generalize the input task.
Generalized Task：Sum of prime numbers from A to B

> Determine if you should make a tool.
I do not need to create a tool to run it.



> Entering new AgentExecutor chain...
I need to find the sum of prime numbers between 8 and 15.
Action: SumOfPrimesTool
Action Input: 8,15
Observation: 24
Thought:I now know the final answer.
Final Answer: 24

> Finished chain.
